Évaluation statistique : régression logistique

L’objet du projet est d’analyser, à partir de l’enquête Conditions de Travail 2013, les déterminants pour qu’une
femme soit le principal apporteur de ressources actuel du ménage. 

Imports

In [31]:
import pandas as pd
import pyreadstat
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.linear_model import LogisticRegression
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.dummy import DummyClassifier
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score

In [105]:
df, meta = pyreadstat.read_sas7bdat("/Users/paulaneira/Desktop/Project 3 statistics/individus_ct2013 (1).sas7bdat")
df

,C1C,C1D,IDENT_MEN,IDENT_IND,NPERS,NACTIFS,TYPMEN5,TYPMEN15,IR,REVMEN,...,lnaisd,lnaisper,lnaismer,natnaisper,natnaismer,nati,revannu_drap,revannu,revmensc,revmenscc_drap
0,2,2,0100019101000,010001910100002,2.0,1.0,2,22,1,2200.0,...,11,11,11,,,,1.0,1500.0,125.000000,1.0
1,,,0100083001000,010008300100002,2.0,2.0,5,52,2,2250.0,...,11,11,11,,,,-2.0,NaN,NaN,-2.0
2,2,2,0100084601000,010008460100002,5.0,2.0,4,42,1,2300.0,...,11,11,11,,,,1.0,0.0,0.000000,1.0
3,2,2,0100091701000,010009170100001,2.0,1.0,3,31,4,NaN,...,11,11,11,,,,1.0,3700.0,308.333333,1.0
4,1,1,0100091801000,010009180100001,1.0,1.0,1,10,1,1200.0,...,11,11,11,,,,1.0,15000.0,1250.000000,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33668,4,3,9300058901000,930005890100001,4.0,2.0,4,42,3,NaN,...,11,11,11,,,,0.0,NaN,NaN,-1.0
33669,2,5,9300076701000,930007670100001,2.0,2.0,3,32,3,NaN,...,11,11,11,,,,0.0,NaN,NaN,-2.0
33670,5,5,9300129001000,930012900100001,1.0,1.0,1,10,1,1061.0,...,11,11,45,,45,,0.0,NaN,1060.000000,1.0
33671,1,2,9400000601000,940000060100002,2.0,2.0,3,32,2,1900.0,...,11,11,11,,,,1.0,11000.0,916.666667,1.0


Explore the data

In [63]:
pd.set_option('display.max_columns', None)
df

,C1C,C1D,IDENT_MEN,IDENT_IND,NPERS,NACTIFS,TYPMEN5,TYPMEN15,IR,REVMEN,REVMENMN,REVMENMX,COEFFUC,REVMENUC,REVMENMNUC,REVMENMXUC,CJSITUA,CJACTOCCUP,SEXE,MNAIS,ANAIS,AGE,AGEQ,LNAIS,DEPNAIS,NAIS7,ANARRIV,AGARRIV,COUPLE,COUPLRP,CONJOINT,ETAMATRI,PACS,ENFANT,LIEN_01,LIEN_02,LIEN_03,LIEN_04,LIEN_05,LIEN_06,LIEN_07,LIEN_08,LIEN_09,LIEN_10,LIEN_11,LIEN_12,LIEN_13,LIEN_14,LIEN_15,LIEN_16,LIEN_17,LIEN_18,LIEN_19,LIEN_20,TYPOLOG,IPROPLOC,AUTLOG,PRACT,IPRAN,LIENPREF,FAMPRINC,SITUA,TRAREF,PASTRA,RABS,RABSP,RABSPU,AIDFAM,INFORM,MER1E,PER1E,NOIK1,NOIK2,CJENQUETE,NBENFM3,NBENF3A17,NBENF18P,TOTREVEN,ITOTREV,TRANCHRE,EREG,NBSAL,NBSALAID,ENCADR,TITPUB,DUDET,DUDETU,DEBDETM,DEBDETA,TPSINT,TPSINTU,RDET,DATANT,DAMOIS,LIEUW,TRAJET,CLIENT,POURCLIEN,ANETA,FRANCE,DRH,DIFPAYE,NBSALA,AUTRETA,NBSALB,REVTRSAL,REVTRNSA,PAYECOM,TPP,TXTPPB,RAISTP,HH,HHTOT,JOURTR,REPOS,SAMEDI,NBSAME,DIMANCHE,NBDIMA,DIMCOMP,DETHOR,HORANGT,PREMOIS,PRESEM,PREJOUR,HORVAR,PERIODE,DEBUTM,FINM,DEBUT2H,DEBUT2M,FIN2H,FIN2M,CONTROLE,PTMATIN,SOIR,NUIT,NBNUIT,NUICOMP,HSUP,HSUPCOMP,ASTREINTE,NBASTR,ASTRCOMP,JOINDRE,COMBIEN,MAISON,TPMAISON,MISSION,CONGE,CONGEU,LIBCONG,ARETMAL,ATMAL,PRISCONG,URGFAM1,CVFVP,AIDHID,CWDEBOU,CWPOSTU,CWDEPLA,CWLOURD,CWMVT,CWVIB,CWVUE,CWLETR,CWMINUS,CWVISO,HYGSAL,HYGCHUMI,HYGCOUR,HYGODEUR,HYGCHAUD,HYGFROID,HYGSANI,HYGVUE,HYGPRIV,CONDUITE,NBRKM,NBRKMU,SECFUPOU,SECTOXNO,SECINFEC,SECACCID,SECROUT,ENTENDR,EPI,EPIUTIL,RISK,RISKINF1,RISKINF2,RISKINF3,RISKINF4,RISKINF5,FORMSEC,DOCEVAL,ASSOEVAL,MEDECIN,VISITUNI,SECURITE,APPLIQ,APPLINSP,ACCIDT,ACTUACC1,ACTUACC2,DATACCM,DATACCA,ARRET,NBJARR,SIGN,INDEMNI,REDRISK,RWDEP,RWCAD,RWTECH,RWCOLEG,RWNORMH,RWNORMJ,RWDEM,RWDEMAND,RWSURV,RWINFO,CHAINE,DEPECH,DEBORD,INTERACT,OBJECTIF,OBJMODIF,OBJATTEIN,DELAIS,PUBLIC,PUBLIC1,PUBLIC2,POLY,POLYQUAL,POLYERR,INTERUP,REMPLAS,COMMENT,STARK,INCIDENT,PROCEDUR,REPETE,CYCLE,MONOTON,NOUVELLE,RISQUAL,RISFINA,RISDANG,RISEMPL,RESTMAI,RESTMAIN,SIEG34,COMPUTIL1,COMPUTIL2,AIDCHEF,AIDCOLL,AIDAUTR,AIDEXT,AIDOM,TRAVSEUL,CHGTCOLL,COLLECT,REUNION,ACCHEF,DICHEF,ACCOL,DICOL,TENSION1,TENSION2,TENSION3,TENSION4,DETRESSE,CALMER,CONTRAD,EVA,EVACRIT,SYNDIC,SYNDICNS,DISCURP,CHSCT,CORRTAN,CORRINF,CORRCOP,CORRCOL,CORRLOG,CORRMAT,CORRFORM,TELEPORT,MICRO1,MICRO2,MEL,INTRANET,MELDOM,RESDOM,INFOH,INFOHU1,INFOHU2,CRAINTE,METIER,NOCHOM,TENIR,SOUHAIT,FORTMOD1,FORTMOD2,FORTMOD3,FORTMOD4,FORTMOD5,FORTMOD6,FORTMOD7,CHANGOP,CHGTINFO,CHGTCONS,CHGTINFL,BSANTE,BCHRO,BLIMI,BRECO,ACTIP,STATUTP,ACTIM,STATUTM,ETUDIPL,FINETUD,FINETUDU,TCHOL,TCHOC,TSAN,TINA,TINDEP,TINTERIM,TCDD,NBEMP,STATUTA,DEMPRO,DEMENA,LANG1,LANG2,LANG4,REPQAA,INTRO,A1A,A1B,A1C,A1D,A1E,A1F,A1G,A1H,A1I,A1J,A1K,A1L,A2A,A2B,A2C,A2D,A2E,A2F,A2G,A2H,A2I,A2J,A2K,A2L,B1A,B1B,B1C,B1D,B1E,B1F,B1G,B1H,B1I,B1J,B2A,B2B,B2C,B2D,B3A,B3B,B3C,B3D,B3E,B3F,B3G,B3H,B4A,B4B,B4C,B4D,B5A,B5B,B5C,B5D,B5E,B5F,B6,C1A,C1B,C1E,C2,CHERCHEUR,V,STATUTEXT,STATUT,AUTSAL,CLASSIF,TYPEMPLOI,FONCTION,AIDE1E,AIDE2E,AFTYPTRAV,ACTIVFIN,SUPH,SUPA,OPA,DEBUTH,FINH,INTERNET,NAIP,NAIM,NATIO1NA,NATIO1NB,ETUDES,FORMINIT,DIPLOME,TOTREVEN_DRAP,DUDET_DRAP,DEBDETM_DRAP,DEBDETA_DRAP,TPSINT_DRAP,DATANT_DRAP,DAMOIS_DRAP,TRAJET_DRAP,POURCLIEN_DRAP,ANETA_DRAP,REVMENS_DRAP,REVMENS,HH_DRAP,HHTOT_DRAP,JOURTR_DRAP,NBSAME_DRAP,NBDIMA_DRAP,DEBUTH_DRAP,DEBUTM_DRAP,FINH_DRAP,FINM_DRAP,DEBUT2H_DRAP,DEBUT2M_DRAP,FIN2H_DRAP,FIN2M_DRAP,NBNUIT_DRAP,NBASTR_DRAP,COMBIEN_DRAP,TPMAISON_DRAP,CONGE_DRAP,ATMAL_DRAP,NBRKM_DRAP,INFOH_DRAP,RESTMAIN_DRAP,DATACCM_DRAP,DATACCA_DRAP,NBJARR_DRAP,CHAMP_EMPL,QUAL_EMPLX,QUAL_ADR,QUAL_SIRET,QUALITE_VOLET,TYPEXTFP,PREVIS,KMANNU,TPSINFO,NATIO,IMMI,PERIMMI,MERIMMI,LIENMIG,AGFINETU,ANFINETU,WHO,NBSALENTC,ELIGEXT,CSE,CSEPER,CSEMER,CSEDEB,PE,PROFPER,PROFMER,PROFDEB,DOUBLONTYPE,NUMFA_SOURCE,SSECH_SOURCE,IDENT_FA_SOURCE,IDENT_LOG_SOURCE,IDENT_MEN_SOURCE,IDENT_IND_SOURCE,RGES,NUMFA,SSECH,LE,EC,BS,IDENT_FA,IDENT_LOG,peun,peun10,pondcal,pondqaa,typrepqaa,duree_cessation,CHAMP_CT2005,militaire,csei,cser,csermer,cserper,cserdeb,fap225,fap87,naf17,naf4,NUMENQ_ANON,region,tps_interim,tps_contrat,anciennete,debut_contrat,co

Missing values

The variable revannu (Revenus nets au cours de la dernière année) and and nati (Nationalité) has 31,418 and 32171 missing values out of 33,673, which is more then  90%. 

This is a very high proportion, therefore  we consider ignoring these variables due to the limited information it provides.

In [64]:
# Check for missing values
missing_values = df.isnull().sum()
missing_values

C1C                   0
C1D                   0
IDENT_MEN             0
IDENT_IND             0
NPERS                 0
                  ...  
nati                  0
revannu_drap          0
revannu           31418
revmensc           3623
revmenscc_drap        0
Length: 541, dtype: int64

In [83]:
# Fill NaN values in numeric columns with the mean of each column
non_numeric_cols = df.select_dtypes(exclude=[np.number]).columns
df[non_numeric_cols] = df[non_numeric_cols].fillna('missing')
df

,C1C,C1D,NPERS,NACTIFS,TYPMEN5,TYPMEN15,IR,REVMEN,REVMENMN,REVMENMX,COEFFUC,REVMENUC,REVMENMNUC,REVMENMXUC,CJSITUA,CJACTOCCUP,SEXE,MNAIS,ANAIS,AGE,AGEQ,LNAIS,DEPNAIS,NAIS7,ANARRIV,AGARRIV,COUPLE,COUPLRP,CONJOINT,ETAMATRI,PACS,ENFANT,LIEN_01,LIEN_02,LIEN_03,LIEN_04,LIEN_05,LIEN_06,LIEN_07,LIEN_08,LIEN_09,LIEN_10,LIEN_11,LIEN_12,LIEN_13,LIEN_14,LIEN_15,LIEN_16,LIEN_17,LIEN_18,LIEN_19,LIEN_20,TYPOLOG,IPROPLOC,AUTLOG,PRACT,IPRAN,LIENPREF,FAMPRINC,SITUA,TRAREF,PASTRA,RABS,RABSP,RABSPU,AIDFAM,INFORM,MER1E,PER1E,NOIK1,NOIK2,CJENQUETE,NBENFM3,NBENF3A17,NBENF18P,TOTREVEN,ITOTREV,TRANCHRE,EREG,NBSAL,NBSALAID,ENCADR,TITPUB,DUDET,DUDETU,DEBDETM,DEBDETA,TPSINT,TPSINTU,RDET,DATANT,DAMOIS,LIEUW,TRAJET,CLIENT,POURCLIEN,ANETA,FRANCE,DRH,DIFPAYE,NBSALA,AUTRETA,NBSALB,REVTRSAL,REVTRNSA,PAYECOM,TPP,TXTPPB,RAISTP,HH,HHTOT,JOURTR,REPOS,SAMEDI,NBSAME,DIMANCHE,NBDIMA,DIMCOMP,DETHOR,HORANGT,PREMOIS,PRESEM,PREJOUR,HORVAR,PERIODE,DEBUTM,FINM,DEBUT2H,DEBUT2M,FIN2H,FIN2M,CONTROLE,PTMATIN,SOIR,NUIT,NBNUIT,NUICOMP,HSUP,HSUPCOMP,ASTREINTE,NBASTR,ASTRCOMP,JOINDRE,COMBIEN,MAISON,TPMAISON,MISSION,CONGE,CONGEU,LIBCONG,ARETMAL,ATMAL,PRISCONG,URGFAM1,CVFVP,AIDHID,CWDEBOU,CWPOSTU,CWDEPLA,CWLOURD,CWMVT,CWVIB,CWVUE,CWLETR,CWMINUS,CWVISO,HYGSAL,HYGCHUMI,HYGCOUR,HYGODEUR,HYGCHAUD,HYGFROID,HYGSANI,HYGVUE,HYGPRIV,CONDUITE,NBRKM,NBRKMU,SECFUPOU,SECTOXNO,SECINFEC,SECACCID,SECROUT,ENTENDR,EPI,EPIUTIL,RISK,RISKINF1,RISKINF2,RISKINF3,RISKINF4,RISKINF5,FORMSEC,DOCEVAL,ASSOEVAL,MEDECIN,VISITUNI,SECURITE,APPLIQ,APPLINSP,ACCIDT,ACTUACC1,ACTUACC2,DATACCM,DATACCA,ARRET,NBJARR,SIGN,INDEMNI,REDRISK,RWDEP,RWCAD,RWTECH,RWCOLEG,RWNORMH,RWNORMJ,RWDEM,RWDEMAND,RWSURV,RWINFO,CHAINE,DEPECH,DEBORD,INTERACT,OBJECTIF,OBJMODIF,OBJATTEIN,DELAIS,PUBLIC,PUBLIC1,PUBLIC2,POLY,POLYQUAL,POLYERR,INTERUP,REMPLAS,COMMENT,STARK,INCIDENT,PROCEDUR,REPETE,CYCLE,MONOTON,NOUVELLE,RISQUAL,RISFINA,RISDANG,RISEMPL,RESTMAI,RESTMAIN,SIEG34,COMPUTIL1,COMPUTIL2,AIDCHEF,AIDCOLL,AIDAUTR,AIDEXT,AIDOM,TRAVSEUL,CHGTCOLL,COLLECT,REUNION,ACCHEF,DICHEF,ACCOL,DICOL,TENSION1,TENSION2,TENSION3,TENSION4,DETRESSE,CALMER,CONTRAD,EVA,EVACRIT,SYNDIC,SYNDICNS,DISCURP,CHSCT,CORRTAN,CORRINF,CORRCOP,CORRCOL,CORRLOG,CORRMAT,CORRFORM,TELEPORT,MICRO1,MICRO2,MEL,INTRANET,MELDOM,RESDOM,INFOH,INFOHU1,INFOHU2,CRAINTE,METIER,NOCHOM,TENIR,SOUHAIT,FORTMOD1,FORTMOD2,FORTMOD3,FORTMOD4,FORTMOD5,FORTMOD6,FORTMOD7,CHANGOP,CHGTINFO,CHGTCONS,CHGTINFL,BSANTE,BCHRO,BLIMI,BRECO,ACTIP,STATUTP,ACTIM,STATUTM,ETUDIPL,FINETUD,FINETUDU,TCHOL,TCHOC,TSAN,TINA,TINDEP,TINTERIM,TCDD,NBEMP,STATUTA,DEMPRO,DEMENA,LANG1,LANG2,LANG4,REPQAA,INTRO,A1A,A1B,A1C,A1D,A1E,A1F,A1G,A1H,A1I,A1J,A1K,A1L,A2A,A2B,A2C,A2D,A2E,A2F,A2G,A2H,A2I,A2J,A2K,A2L,B1A,B1B,B1C,B1D,B1E,B1F,B1G,B1H,B1I,B1J,B2A,B2B,B2C,B2D,B3A,B3B,B3C,B3D,B3E,B3F,B3G,B3H,B4A,B4B,B4C,B4D,B5A,B5B,B5C,B5D,B5E,B5F,B6,C1A,C1B,C1E,C2,CHERCHEUR,V,STATUTEXT,STATUT,AUTSAL,CLASSIF,TYPEMPLOI,FONCTION,AIDE1E,AIDE2E,AFTYPTRAV,ACTIVFIN,SUPH,SUPA,OPA,DEBUTH,FINH,INTERNET,NAIP,NAIM,NATIO1NA,NATIO1NB,ETUDES,FORMINIT,DIPLOME,TOTREVEN_DRAP,DUDET_DRAP,DEBDETM_DRAP,DEBDETA_DRAP,TPSINT_DRAP,DATANT_DRAP,DAMOIS_DRAP,TRAJET_DRAP,POURCLIEN_DRAP,ANETA_DRAP,REVMENS_DRAP,REVMENS,HH_DRAP,HHTOT_DRAP,JOURTR_DRAP,NBSAME_DRAP,NBDIMA_DRAP,DEBUTH_DRAP,DEBUTM_DRAP,FINH_DRAP,FINM_DRAP,DEBUT2H_DRAP,DEBUT2M_DRAP,FIN2H_DRAP,FIN2M_DRAP,NBNUIT_DRAP,NBASTR_DRAP,COMBIEN_DRAP,TPMAISON_DRAP,CONGE_DRAP,ATMAL_DRAP,NBRKM_DRAP,INFOH_DRAP,RESTMAIN_DRAP,DATACCM_DRAP,DATACCA_DRAP,NBJARR_DRAP,CHAMP_EMPL,QUAL_EMPLX,QUAL_ADR,QUAL_SIRET,QUALITE_VOLET,TYPEXTFP,PREVIS,KMANNU,TPSINFO,NATIO,IMMI,PERIMMI,MERIMMI,LIENMIG,AGFINETU,ANFINETU,WHO,NBSALENTC,ELIGEXT,CSE,CSEPER,CSEMER,CSEDEB,PE,PROFPER,PROFMER,PROFDEB,DOUBLONTYPE,NUMFA_SOURCE,SSECH_SOURCE,IDENT_FA_SOURCE,IDENT_LOG_SOURCE,IDENT_MEN_SOURCE,IDENT_IND_SOURCE,RGES,NUMFA,SSECH,LE,EC,BS,IDENT_FA,IDENT_LOG,peun,peun10,pondcal,pondqaa,typrepqaa,duree_cessation,CHAMP_CT2005,militaire,csei,cser,csermer,cserper,cserdeb,fap225,fap87,naf17,naf4,NUMENQ_ANON,region,tps_interim,tps_contrat,anciennete,debut_contrat,congetr,naf38,lnaisd,l

In [84]:
# For non-numeric columns fill NaN values with a placeholder or mode
non_numeric_cols = df.select_dtypes(exclude=[np.number]).columns
df[non_numeric_cols] = df[non_numeric_cols].fillna('missing')
df

,C1C,C1D,NPERS,NACTIFS,TYPMEN5,TYPMEN15,IR,REVMEN,REVMENMN,REVMENMX,COEFFUC,REVMENUC,REVMENMNUC,REVMENMXUC,CJSITUA,CJACTOCCUP,SEXE,MNAIS,ANAIS,AGE,AGEQ,LNAIS,DEPNAIS,NAIS7,ANARRIV,AGARRIV,COUPLE,COUPLRP,CONJOINT,ETAMATRI,PACS,ENFANT,LIEN_01,LIEN_02,LIEN_03,LIEN_04,LIEN_05,LIEN_06,LIEN_07,LIEN_08,LIEN_09,LIEN_10,LIEN_11,LIEN_12,LIEN_13,LIEN_14,LIEN_15,LIEN_16,LIEN_17,LIEN_18,LIEN_19,LIEN_20,TYPOLOG,IPROPLOC,AUTLOG,PRACT,IPRAN,LIENPREF,FAMPRINC,SITUA,TRAREF,PASTRA,RABS,RABSP,RABSPU,AIDFAM,INFORM,MER1E,PER1E,NOIK1,NOIK2,CJENQUETE,NBENFM3,NBENF3A17,NBENF18P,TOTREVEN,ITOTREV,TRANCHRE,EREG,NBSAL,NBSALAID,ENCADR,TITPUB,DUDET,DUDETU,DEBDETM,DEBDETA,TPSINT,TPSINTU,RDET,DATANT,DAMOIS,LIEUW,TRAJET,CLIENT,POURCLIEN,ANETA,FRANCE,DRH,DIFPAYE,NBSALA,AUTRETA,NBSALB,REVTRSAL,REVTRNSA,PAYECOM,TPP,TXTPPB,RAISTP,HH,HHTOT,JOURTR,REPOS,SAMEDI,NBSAME,DIMANCHE,NBDIMA,DIMCOMP,DETHOR,HORANGT,PREMOIS,PRESEM,PREJOUR,HORVAR,PERIODE,DEBUTM,FINM,DEBUT2H,DEBUT2M,FIN2H,FIN2M,CONTROLE,PTMATIN,SOIR,NUIT,NBNUIT,NUICOMP,HSUP,HSUPCOMP,ASTREINTE,NBASTR,ASTRCOMP,JOINDRE,COMBIEN,MAISON,TPMAISON,MISSION,CONGE,CONGEU,LIBCONG,ARETMAL,ATMAL,PRISCONG,URGFAM1,CVFVP,AIDHID,CWDEBOU,CWPOSTU,CWDEPLA,CWLOURD,CWMVT,CWVIB,CWVUE,CWLETR,CWMINUS,CWVISO,HYGSAL,HYGCHUMI,HYGCOUR,HYGODEUR,HYGCHAUD,HYGFROID,HYGSANI,HYGVUE,HYGPRIV,CONDUITE,NBRKM,NBRKMU,SECFUPOU,SECTOXNO,SECINFEC,SECACCID,SECROUT,ENTENDR,EPI,EPIUTIL,RISK,RISKINF1,RISKINF2,RISKINF3,RISKINF4,RISKINF5,FORMSEC,DOCEVAL,ASSOEVAL,MEDECIN,VISITUNI,SECURITE,APPLIQ,APPLINSP,ACCIDT,ACTUACC1,ACTUACC2,DATACCM,DATACCA,ARRET,NBJARR,SIGN,INDEMNI,REDRISK,RWDEP,RWCAD,RWTECH,RWCOLEG,RWNORMH,RWNORMJ,RWDEM,RWDEMAND,RWSURV,RWINFO,CHAINE,DEPECH,DEBORD,INTERACT,OBJECTIF,OBJMODIF,OBJATTEIN,DELAIS,PUBLIC,PUBLIC1,PUBLIC2,POLY,POLYQUAL,POLYERR,INTERUP,REMPLAS,COMMENT,STARK,INCIDENT,PROCEDUR,REPETE,CYCLE,MONOTON,NOUVELLE,RISQUAL,RISFINA,RISDANG,RISEMPL,RESTMAI,RESTMAIN,SIEG34,COMPUTIL1,COMPUTIL2,AIDCHEF,AIDCOLL,AIDAUTR,AIDEXT,AIDOM,TRAVSEUL,CHGTCOLL,COLLECT,REUNION,ACCHEF,DICHEF,ACCOL,DICOL,TENSION1,TENSION2,TENSION3,TENSION4,DETRESSE,CALMER,CONTRAD,EVA,EVACRIT,SYNDIC,SYNDICNS,DISCURP,CHSCT,CORRTAN,CORRINF,CORRCOP,CORRCOL,CORRLOG,CORRMAT,CORRFORM,TELEPORT,MICRO1,MICRO2,MEL,INTRANET,MELDOM,RESDOM,INFOH,INFOHU1,INFOHU2,CRAINTE,METIER,NOCHOM,TENIR,SOUHAIT,FORTMOD1,FORTMOD2,FORTMOD3,FORTMOD4,FORTMOD5,FORTMOD6,FORTMOD7,CHANGOP,CHGTINFO,CHGTCONS,CHGTINFL,BSANTE,BCHRO,BLIMI,BRECO,ACTIP,STATUTP,ACTIM,STATUTM,ETUDIPL,FINETUD,FINETUDU,TCHOL,TCHOC,TSAN,TINA,TINDEP,TINTERIM,TCDD,NBEMP,STATUTA,DEMPRO,DEMENA,LANG1,LANG2,LANG4,REPQAA,INTRO,A1A,A1B,A1C,A1D,A1E,A1F,A1G,A1H,A1I,A1J,A1K,A1L,A2A,A2B,A2C,A2D,A2E,A2F,A2G,A2H,A2I,A2J,A2K,A2L,B1A,B1B,B1C,B1D,B1E,B1F,B1G,B1H,B1I,B1J,B2A,B2B,B2C,B2D,B3A,B3B,B3C,B3D,B3E,B3F,B3G,B3H,B4A,B4B,B4C,B4D,B5A,B5B,B5C,B5D,B5E,B5F,B6,C1A,C1B,C1E,C2,CHERCHEUR,V,STATUTEXT,STATUT,AUTSAL,CLASSIF,TYPEMPLOI,FONCTION,AIDE1E,AIDE2E,AFTYPTRAV,ACTIVFIN,SUPH,SUPA,OPA,DEBUTH,FINH,INTERNET,NAIP,NAIM,NATIO1NA,NATIO1NB,ETUDES,FORMINIT,DIPLOME,TOTREVEN_DRAP,DUDET_DRAP,DEBDETM_DRAP,DEBDETA_DRAP,TPSINT_DRAP,DATANT_DRAP,DAMOIS_DRAP,TRAJET_DRAP,POURCLIEN_DRAP,ANETA_DRAP,REVMENS_DRAP,REVMENS,HH_DRAP,HHTOT_DRAP,JOURTR_DRAP,NBSAME_DRAP,NBDIMA_DRAP,DEBUTH_DRAP,DEBUTM_DRAP,FINH_DRAP,FINM_DRAP,DEBUT2H_DRAP,DEBUT2M_DRAP,FIN2H_DRAP,FIN2M_DRAP,NBNUIT_DRAP,NBASTR_DRAP,COMBIEN_DRAP,TPMAISON_DRAP,CONGE_DRAP,ATMAL_DRAP,NBRKM_DRAP,INFOH_DRAP,RESTMAIN_DRAP,DATACCM_DRAP,DATACCA_DRAP,NBJARR_DRAP,CHAMP_EMPL,QUAL_EMPLX,QUAL_ADR,QUAL_SIRET,QUALITE_VOLET,TYPEXTFP,PREVIS,KMANNU,TPSINFO,NATIO,IMMI,PERIMMI,MERIMMI,LIENMIG,AGFINETU,ANFINETU,WHO,NBSALENTC,ELIGEXT,CSE,CSEPER,CSEMER,CSEDEB,PE,PROFPER,PROFMER,PROFDEB,DOUBLONTYPE,NUMFA_SOURCE,SSECH_SOURCE,IDENT_FA_SOURCE,IDENT_LOG_SOURCE,IDENT_MEN_SOURCE,IDENT_IND_SOURCE,RGES,NUMFA,SSECH,LE,EC,BS,IDENT_FA,IDENT_LOG,peun,peun10,pondcal,pondqaa,typrepqaa,duree_cessation,CHAMP_CT2005,militaire,csei,cser,csermer,cserper,cserdeb,fap225,fap87,naf17,naf4,NUMENQ_ANON,region,tps_interim,tps_contrat,anciennete,debut_contrat,congetr,naf38,lnaisd,l

In [7]:
df.drop(columns=['IDENT_MEN', 'IDENT_IND',], inplace=True)
df

,C1C,C1D,NPERS,NACTIFS,TYPMEN5,TYPMEN15,IR,REVMEN,REVMENMN,REVMENMX,...,lnaisd,lnaisper,lnaismer,natnaisper,natnaismer,nati,revannu_drap,revannu,revmensc,revmenscc_drap
0,2,2,2.0,1.0,2,22,1,2200.0,2200.0,2200.0,...,11,11,11,,,,1.0,1500.0,125.000000,1.0
1,,,2.0,2.0,5,52,2,2250.0,2000.0,2499.0,...,11,11,11,,,,-2.0,NaN,NaN,-2.0
2,2,2,5.0,2.0,4,42,1,2300.0,2300.0,2300.0,...,11,11,11,,,,1.0,0.0,0.000000,1.0
3,2,2,2.0,1.0,3,31,4,NaN,NaN,NaN,...,11,11,11,,,,1.0,3700.0,308.333333,1.0
4,1,1,1.0,1.0,1,10,1,1200.0,1200.0,1200.0,...,11,11,11,,,,1.0,15000.0,1250.000000,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33668,4,3,4.0,2.0,4,42,3,NaN,NaN,NaN,...,11,11,11,,,,0.0,NaN,NaN,-1.0
33669,2,5,2.0,2.0,3,32,3,NaN,NaN,NaN,...,11,11,11,,,,0.0,NaN,NaN,-2.0
33670,5,5,1.0,1.0,1,10,1,1061.0,1061.0,1061.0,...,11,11,45,,45,,0.0,NaN,1060.000000,1.0
33671,1,2,2.0,2.0,3,32,2,1900.0,1800.0,1999.0,...,11,11,11,,,,1.0,11000.0,916.666667,1.0


Data Preprocessing

In [106]:
#Filter for couples living together in the same home

df_en_couple = df[(df['COUPLE']=='1')]
df_en_couple.reset_index(drop=True, inplace=True)
df_en_couple

,C1C,C1D,IDENT_MEN,IDENT_IND,NPERS,NACTIFS,TYPMEN5,TYPMEN15,IR,REVMEN,...,lnaisd,lnaisper,lnaismer,natnaisper,natnaismer,nati,revannu_drap,revannu,revmensc,revmenscc_drap
0,2,2,0100084601000,010008460100002,5.0,2.0,4,42,1,2300.0,...,11,11,11,,,,1.0,0.0,0.000000,1.0
1,2,2,0100091701000,010009170100001,2.0,1.0,3,31,4,NaN,...,11,11,11,,,,1.0,3700.0,308.333333,1.0
2,,,0100097401000,010009740100002,4.0,2.0,4,42,2,1650.0,...,11,11,11,,,,-2.0,NaN,NaN,-2.0
3,2,2,0100098501000,010009850100001,2.0,2.0,3,32,1,2200.0,...,11,11,11,,,,1.0,15000.0,1250.000000,1.0
4,,,0100098601000,010009860100001,4.0,1.0,4,41,1,673.0,...,11,11,11,,,,1.0,0.0,0.000000,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24019,3,2,9300012703000,930001270300001,2.0,2.0,3,32,1,2600.0,...,11,21,21,11,11,,0.0,NaN,1400.000000,1.0
24020,4,3,9300058901000,930005890100001,4.0,2.0,4,42,3,NaN,...,11,11,11,,,,0.0,NaN,NaN,-1.0
24021,2,5,9300076701000,930007670100001,2.0,2.0,3,32,3,NaN,...,11,11,11,,,,0.0,NaN,NaN,-2.0
24022,1,2,9400000601000,940000060100002,2.0,2.0,3,32,2,1900.0,...,11,11,11,,,,1.0,11000.0,916.666667,1.0


In [107]:
# Create dependent variable fprov from variable sex and pract
df_en_couple.loc[:, 'fprov'] = (((df_en_couple['PRACT'] == "1") & (df_en_couple['SEXE'] == "2"))| ((df_en_couple['PRACT'] == "2") & (df_en_couple['SEXE'] == "1"))).astype(int)
df_en_couple

/var/folders/nd/tlvj5bgn4zv2nrczbbjppffr0000gn/T/ipykernel_871/3148408508.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_en_couple.loc[:, 'fprov'] = (((df_en_couple['PRACT'] == "1") & (df_en_couple['SEXE'] == "2"))| ((df_en_couple['PRACT'] == "2") & (df_en_couple['SEXE'] == "1"))).astype(int)


,C1C,C1D,IDENT_MEN,IDENT_IND,NPERS,NACTIFS,TYPMEN5,TYPMEN15,IR,REVMEN,...,lnaisper,lnaismer,natnaisper,natnaismer,nati,revannu_drap,revannu,revmensc,revmenscc_drap,fprov
0,2,2,0100084601000,010008460100002,5.0,2.0,4,42,1,2300.0,...,11,11,,,,1.0,0.0,0.000000,1.0,0
1,2,2,0100091701000,010009170100001,2.0,1.0,3,31,4,NaN,...,11,11,,,,1.0,3700.0,308.333333,1.0,0
2,,,0100097401000,010009740100002,4.0,2.0,4,42,2,1650.0,...,11,11,,,,-2.0,NaN,NaN,-2.0,0
3,2,2,0100098501000,010009850100001,2.0,2.0,3,32,1,2200.0,...,11,11,,,,1.0,15000.0,1250.000000,1.0,1
4,,,0100098601000,010009860100001,4.0,1.0,4,41,1,673.0,...,11,11,,,,1.0,0.0,0.000000,1.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24019,3,2,9300012703000,930001270300001,2.0,2.0,3,32,1,2600.0,...,21,21,11,11,,0.0,NaN,1400.000000,1.0,0
24020,4,3,9300058901000,930005890100001,4.0,2.0,4,42,3,NaN,...,11,11,,,,0.0,NaN,NaN,-1.0,0
24021,2,5,9300076701000,930007670100001,2.0,2.0,3,32,3,NaN,...,11,11,,,,0.0,NaN,NaN,-2.0,1
24022,1,2,9400000601000,940000060100002,2.0,2.0,3,32,2,1900.0,...,11,11,,,,1.0,11000.0,916.666667,1.0,0


In [98]:
df_en_couple[['PRACT', 'SEXE', 'fprov']]

,PRACT,SEXE,fprov
0,1,1,0
1,2,1,1
2,1,1,0
3,1,1,0
4,1,2,1
...,...,...,...
8672,1,1,0
8673,1,1,0
8674,1,2,1
8675,2,2,0


In [1]:
# fprov_counts = df_en_couple['fprov'].value_counts()
femme_contributeur = (df_en_couple['fprov'] == 1).sum()
homme_contributeur = (df_en_couple['fprov'] == 0).sum()
total = homme_contributeur + femme_contributeur
print(f"{femme_contributeur} ({100*femme_contributeur/total:.2f}%) couples dont la femme est le principal contributeur du foyer")
print(f"{homme_contributeur} ({100*homme_contributeur/total:.2f}%) couples dont la femme n'est pas le principal contributeur du foyer")


NameError: name 'df_en_couple' is not defined

Data distribution

In [117]:
df2= df_en_couple[['SEXE', 'PRACT', 'fprov','SITUA', 'PASTRA','TRAREF', 'RABS', 'AIDFAM', 'IPRAN', 'TYPMEN5', 'TOTREVEN', 'REVMEN', 'STATUT', 'revannu', 'AGE']]
df2

,SEXE,PRACT,fprov,SITUA,PASTRA,TRAREF,RABS,AIDFAM,IPRAN,TYPMEN5,TOTREVEN,REVMEN,STATUT,revannu,AGE
0,2,2,0,4,2,2,,1,2,4,2300.0,2300.0,8,0.0,49.0
1,1,1,0,1,,1,,,2,3,300.0,NaN,10,3700.0,42.0
2,1,1,0,1,,1,,,2,4,NaN,1650.0,10,NaN,41.0
3,1,2,1,1,,1,,,2,3,2200.0,2200.0,10,15000.0,64.0
4,1,1,0,1,,1,,,2,4,673.0,673.0,10,0.0,50.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24019,1,1,0,1,,1,,,2,3,2600.0,2600.0,2,NaN,44.0
24020,1,1,0,1,,1,,,2,4,NaN,NaN,6,NaN,50.0
24021,2,1,1,1,,1,,,2,3,NaN,NaN,6,NaN,51.0
24022,1,1,0,1,,1,,,2,3,NaN,1900.0,10,11000.0,58.0


In [122]:
df2['SEXE'] = df2['SEXE'].astype('category').cat.codes
df2['SITUA'] = df2['SITUA'].astype('category').cat.codes
df2['STATUT'] = df2['STATUT'].astype('category').cat.codes
df2['revenu'] = pd.to_numeric(df2['revannu'], errors='coerce')
df2['AGE'] = pd.to_numeric(df2['AGE'], errors='coerce')
df2['TOTREVEN'] = pd.to_numeric(df2['TOTREVEN'], errors='coerce')


df2 = df2.apply(pd.to_numeric, errors='coerce')
df2.fillna(df2.mean(), inplace=True)
